## AutoML

In [1]:
%pip install autogluon.tabular[all] -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\lucas\\Repos\\real-time-process-prediction\\venv\\Lib\\site-packages\\~sutil\\_psutil_windows.pyd'
Check the permissions.

You should consider upgrading via the 'c:\Users\lucas\Repos\real-time-process-prediction\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd

# AutoML
from autogluon.tabular import TabularDataset, TabularPredictor

c:\Users\lucas\Repos\real-time-process-prediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
X_train = pd.read_csv('data/processed/X_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
X_val = pd.read_csv('data/processed/X_val.csv')
y_val = pd.read_csv('data/processed/y_val.csv')
X_test = pd.read_csv('data/processed/X_test.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

In [6]:
train_set = pd.concat([X_train, y_train], axis=1)
valid_set = pd.concat([X_val, y_val], axis=1)
test_set = pd.concat([X_test, y_test], axis=1)

In [8]:
train_set = TabularDataset(train_set)
train_set.head()

,vazaoVapor,pressaoVapor,temperaturaVapor,cargaVaporTG1,cargaVaporTG2,habilitaTG1,habilitaTG2,consumoEspecificoTG1_1,consumoEspecificoTG1_2,consumoEspecificoTG2_1,consumoEspecificoTG2_2,potenciaGeradaTG1_1,potenciaGeradaTG1_2,potenciaGeradaTG2_1,potenciaGeradaTG2_2,vazaoVaporEscape
0,376.684774,57.704084,823.608926,187.716124,63.247392,1,1,8.508188,9.357013,4.639457,11.947866,22.062996,20.061545,13.632499,3.619675,377.273956
1,283.934340,60.319730,867.005162,205.307873,85.807646,0,1,0.000000,0.000000,4.058899,11.679906,0.000000,0.000000,19.371380,5.019430,307.311031
2,406.481097,61.930230,755.382943,147.415838,104.507646,1,0,15.841750,8.632192,0.000000,0.000000,9.305527,17.077451,0.000000,0.000000,370.484139
3,396.502754,60.383351,853.381528,168.249068,58.854856,1,1,7.458438,9.194291,4.336866,11.990789,22.558217,18.299297,13.570827,3.240392,422.212430
4,278.490964,63.136766,824.400477,123.498988,110.203105,1,0,10.579387,7.751438,0.000000,0.000000,11.673549,15.932396,0.000000,0.000000,210.498993


## Model train

Index(['consumoEspecificoTG1_1', 'consumoEspecificoTG1_2',
       'consumoEspecificoTG2_1', 'consumoEspecificoTG2_2',
       'potenciaGeradaTG1_1', 'potenciaGeradaTG1_2', 'potenciaGeradaTG2_1',
       'potenciaGeradaTG2_2', 'vazaoVaporEscape'],
      dtype='object')

In [14]:
target = y_test.columns.tolist()
target

['consumoEspecificoTG1_1',
 'consumoEspecificoTG1_2',
 'consumoEspecificoTG2_1',
 'consumoEspecificoTG2_2',
 'potenciaGeradaTG1_1',
 'potenciaGeradaTG1_2',
 'potenciaGeradaTG2_1',
 'potenciaGeradaTG2_2',
 'vazaoVaporEscape']

## Multilabel

Train a model for each target column.

In [20]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor:
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = os.path.join(self.path, "Predictor_" + str(label))
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=os.path.join(self.path, self.multi_predictor_file), object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        return load_pkl.load(path=os.path.join(path, cls.multi_predictor_file))

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [22]:
len(target)

9

In [23]:
eval_metrics = ['mean_squared_error','mean_absolute_percentage_error', 'r2', 'symmetric_mean_absolute_percentage_error']  # metrics used to evaluate predictions for each label (optional)
save_path = 'agModels'  # specifies folder to store trained models (optional)

time_limit = 180  # how many seconds to train the TabularPredictor for each label, set much larger in your applications!

In [27]:
['regression']*len(target)

['regression',
 'regression',
 'regression',
 'regression',
 'regression',
 'regression',
 'regression',
 'regression',
 'regression']

In [31]:
multi_predictor = MultilabelPredictor(labels=target, problem_types=['regression']*len(target),
                                        eval_metrics=['symmetric_mean_absolute_percentage_error']*len(target), path=save_path)

In [33]:
multi_predictor.fit(train_set, time_limit=time_limit)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       1.89 GB / 15.69 GB (12.0%)
Disk Space Avail:   487.41 GB / 929.98 GB (52.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


Fitting TabularPredictor for label: consumoEspecificoTG1_1 ...


Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "agModels\Predictor_consumoEspecificoTG1_1"
Train Data Rows:    259142
Train Data Columns: 7
Label Column:       consumoEspecificoTG1_1
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1906.77 MB
	Train Data (Original)  Memory Usage: 13.84 MB (0.7% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGen

Fitting TabularPredictor for label: consumoEspecificoTG1_2 ...


	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 6 | ['vazaoVapor', 'pressaoVapor', 'temperaturaVapor', 'cargaVaporTG1', 'cargaVaporTG2', ...]
		('int', [])   : 2 | ['habilitaTG1', 'habilitaTG2']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 6 | ['vazaoVapor', 'pressaoVapor', 'temperaturaVapor', 'cargaVaporTG1', 'cargaVaporTG2', ...]
		('int', ['bool']) : 2 | ['habilitaTG1', 'habilitaTG2']
	0.3s = Fit runtime
	8 features in original data used to generate 8 features in processed data.
	Train Data (Processed) Memory Usage: 12.36 MB (0.6% of available memory)
Data preprocessing and feature engineering runtime = 0.33s ...
AutoGluon will gauge predictive performance using evaluation metric: 'symmetric_mean_absolute_percentage_error'
	

...

[1000]	valid_set's l2: 6.5434	valid_set's symmetric_mean_absolute_percentage_error: -0.00298393
[2000]	valid_set's l2: 4.33902	valid_set's symmetric_mean_absolute_percentage_error: -0.00240899
[3000]	valid_set's l2: 3.46713	valid_set's symmetric_mean_absolute_percentage_error: -0.00214287
[4000]	valid_set's l2: 2.99013	valid_set's symmetric_mean_absolute_percentage_error: -0.00198176
[5000]	valid_set's l2: 2.67227	valid_set's symmetric_mean_absolute_percentage_error: -0.00186482
[6000]	valid_set's l2: 2.43947	valid_set's symmetric_mean_absolute_percentage_error: -0.00177356
[7000]	valid_set's l2: 2.25649	valid_set's symmetric_mean_absolute_percentage_error: -0.00170117
[8000]	valid_set's l2: 2.1225	valid_set's symmetric_mean_absolute_percentage_error: -0.00164376
[9000]	valid_set's l2: 2.01507	valid_set's symmetric_mean_absolute_percentage_error: -0.00159401
[10000]	valid_set's l2: 1.91375	valid_set's symmetric_mean_absolute_percentage_error: -0.00154867


	-0.0015	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	89.64s	 = Training   runtime
	0.68s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 85.71s of the 85.71s of remaining time.


[1000]	valid_set's l2: 3.16449	valid_set's symmetric_mean_absolute_percentage_error: -0.00204797
[2000]	valid_set's l2: 2.19809	valid_set's symmetric_mean_absolute_percentage_error: -0.00167636
[3000]	valid_set's l2: 1.83413	valid_set's symmetric_mean_absolute_percentage_error: -0.00151617
[4000]	valid_set's l2: 1.63773	valid_set's symmetric_mean_absolute_percentage_error: -0.00142586
[5000]	valid_set's l2: 1.5029	valid_set's symmetric_mean_absolute_percentage_error: -0.00135827
[6000]	valid_set's l2: 1.41303	valid_set's symmetric_mean_absolute_percentage_error: -0.00131381
[7000]	valid_set's l2: 1.33882	valid_set's symmetric_mean_absolute_percentage_error: -0.00127673
[8000]	valid_set's l2: 1.2786	valid_set's symmetric_mean_absolute_percentage_error: -0.00124043
[9000]	valid_set's l2: 1.23152	valid_set's symmetric_mean_absolute_percentage_error: -0.001214
[10000]	valid_set's l2: 1.19067	valid_set's symmetric_mean_absolute_percentage_error: -0.00119316


	-0.0012	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	45.03s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 40.03s of the 40.02s of remaining time.
	-0.0031	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	24.59s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 15.33s of the 15.33s of remaining time.
	Ran out of time, early stopping on iteration 1240.
	-0.0018	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	15.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 179.58s of the -0.18s of remaining time.
	Ensemble Weights: {'LightGBM': 0.526, 'LightGBMXT': 0.263, 'CatBoost': 0.158, 'RandomForestMSE': 0.053}
	-0.001	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	0.04s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon tr

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels')


In [38]:
multi_predictor = MultilabelPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained multilabel predictor from file

predictions = multi_predictor.predict(X_test)
print("Predictions:  \n", predictions)

Predicting with TabularPredictor for label: consumoEspecificoTG1_1 ...
Predicting with TabularPredictor for label: consumoEspecificoTG1_2 ...
Predicting with TabularPredictor for label: consumoEspecificoTG2_1 ...
Predicting with TabularPredictor for label: consumoEspecificoTG2_2 ...
Predicting with TabularPredictor for label: potenciaGeradaTG1_1 ...
Predicting with TabularPredictor for label: potenciaGeradaTG1_2 ...
Predicting with TabularPredictor for label: potenciaGeradaTG2_1 ...
Predicting with TabularPredictor for label: potenciaGeradaTG2_2 ...
Predicting with TabularPredictor for label: vazaoVaporEscape ...
Predictions:  
        consumoEspecificoTG1_1  consumoEspecificoTG1_2  consumoEspecificoTG2_1  \
0                    9.700453                8.846820                0.000000   
1                    0.000000                0.000000                6.474423   
2                   50.389988                8.576043                7.690220   
3                    0.000000          

In [40]:
evaluations = multi_predictor.evaluate(test_set)
print(evaluations)
print("Evaluated using metrics:", multi_predictor.eval_metrics)

Evaluating TabularPredictor for label: consumoEspecificoTG1_1 ...
Evaluating TabularPredictor for label: consumoEspecificoTG1_2 ...
Evaluating TabularPredictor for label: consumoEspecificoTG2_1 ...
Evaluating TabularPredictor for label: consumoEspecificoTG2_2 ...
Evaluating TabularPredictor for label: potenciaGeradaTG1_1 ...
Evaluating TabularPredictor for label: potenciaGeradaTG1_2 ...
Evaluating TabularPredictor for label: potenciaGeradaTG2_1 ...
Evaluating TabularPredictor for label: potenciaGeradaTG2_2 ...
Evaluating TabularPredictor for label: vazaoVaporEscape ...
{'consumoEspecificoTG1_1': {'symmetric_mean_absolute_percentage_error': -0.000984814098113492, 'root_mean_squared_error': -1.8097305693955579, 'mean_squared_error': -3.2751247338047698, 'mean_absolute_error': -0.08314428241326301, 'r2': 0.9831010392146552, 'pearsonr': 0.9915561692919219, 'median_absolute_error': -0.00027329772949258313}, 'consumoEspecificoTG1_2': {'symmetric_mean_absolute_percentage_error': -1.2957774023

In [46]:
pd.json_normalize(evaluations,)

,consumoEspecificoTG1_1.symmetric_mean_absolute_percentage_error,consumoEspecificoTG1_1.root_mean_squared_error,consumoEspecificoTG1_1.mean_squared_error,consumoEspecificoTG1_1.mean_absolute_error,consumoEspecificoTG1_1.r2,consumoEspecificoTG1_1.pearsonr,consumoEspecificoTG1_1.median_absolute_error,consumoEspecificoTG1_2.symmetric_mean_absolute_percentage_error,consumoEspecificoTG1_2.root_mean_squared_error,consumoEspecificoTG1_2.mean_squared_error,...,potenciaGeradaTG2_2.r2,potenciaGeradaTG2_2.pearsonr,potenciaGeradaTG2_2.median_absolute_error,vazaoVaporEscape.symmetric_mean_absolute_percentage_error,vazaoVaporEscape.root_mean_squared_error,vazaoVaporEscape.mean_squared_error,vazaoVaporEscape.mean_absolute_error,vazaoVaporEscape.r2,vazaoVaporEscape.pearsonr,vazaoVaporEscape.median_absolute_error
0,-0.000985,-1.809731,-3.275125,-0.083144,0.983101,0.991556,-0.000273,-0.000001,-0.000573,-3.282177e-07,...,0.997147,0.998582,-0.0,-0.001188,-1.093755,-1.196301,-0.754647,0.999752,0.999876,-0.546494


In [60]:
# Extract the best model name for each label
model_map = {}
for label in target:
    predictor = multi_predictor.get_predictor(label)
    best_model_name = predictor.leaderboard()[predictor.leaderboard()['model'] != 'WeightedEnsemble_L2']['model'].iloc[0]
    model_map[label] = best_model_name

In [61]:
model_map

{'consumoEspecificoTG1_1': 'RandomForestMSE',
 'consumoEspecificoTG1_2': 'RandomForestMSE',
 'consumoEspecificoTG2_1': 'RandomForestMSE',
 'consumoEspecificoTG2_2': 'RandomForestMSE',
 'potenciaGeradaTG1_1': 'RandomForestMSE',
 'potenciaGeradaTG1_2': 'RandomForestMSE',
 'potenciaGeradaTG2_1': 'RandomForestMSE',
 'potenciaGeradaTG2_2': 'KNeighborsDist',
 'vazaoVaporEscape': 'LightGBM'}

In [1]:
# import RandomForestMSE from autogluon


## Random Forest

As the AutoML approach showed that the Random Forest model is the best model for most of the target columns, we decided to train a single Random Forest model for all target columns.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [12]:
light_rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbose=2)

In [13]:
light_rf_model.fit(X_train_scaled, y_train, )


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.8s


building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   24.9s finished


RandomForestRegressor(n_jobs=-1, random_state=42, verbose=2)

In [15]:
# Save the model
import joblib
joblib.dump(light_rf_model, 'models/light_rf_model.pkl')

['models/light_rf_model.pkl']

In [17]:
y_pred = light_rf_model.predict(X_test_scaled)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished


In [18]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error
import numpy as np

In [19]:
def symetric_mape(A, F):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE) metric.

    Parameters
    ----------
    A : np.array
        Actual values.

    F : np.array
        Forecasted values.

    Returns
    -------
    float
        SMAPE metric value.
    
    """
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [22]:
# Save scaler
joblib.dump(scaler, 'models/scaler_rf.pkl')

['models/scaler_rf.pkl']

In [20]:
print('Overall metrics:')
print('R2:', r2_score(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('Symetric MAPE:', np.mean(symetric_mape(y_test, y_pred)))
print('MSE:', np.mean((y_test - y_pred)**2))

Overall metrics:
R2: 0.9877542417596251
MAPE: 9384521327512.895
Symetric MAPE: 1.3095146592858176
MSE: 2.845370329345731


c:\Users\lucas\Repos\real-time-process-prediction\venv\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [24]:
targets = ['consumoEspecificoTG1_1', 'consumoEspecificoTG1_2',
           'consumoEspecificoTG2_1', 'consumoEspecificoTG2_2',
           'potenciaGeradaTG1_1', 'potenciaGeradaTG1_2',
           'potenciaGeradaTG2_1', 'potenciaGeradaTG2_2',
           'vazaoVaporEscape']

In [25]:
smape = np.array([symetric_mape(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
mape = np.array([mean_absolute_percentage_error(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
r_2 = np.array([r2_score(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
mse = np.array([np.mean((y_test[target] - y_pred[:, i])**2) for i, target in enumerate(targets)])

# Create a DataFrame with the MAPE for each target
mape_df = pd.DataFrame({'mape': mape, 'smape': smape, 'r2': r_2, 'mse': mse}, index=targets).sort_values('mse')
mape_df

,mape,smape,r2,mse
consumoEspecificoTG1_2,4.183591e-03,0.418133,0.999725,0.006295
potenciaGeradaTG2_2,1.156621e+13,2.233557,0.997853,0.019510
consumoEspecificoTG2_1,1.996693e+13,1.449115,0.996059,0.030902
consumoEspecificoTG2_2,2.945339e+13,0.980052,0.998560,0.048937
potenciaGeradaTG1_2,7.233409e-03,0.721831,0.999128,0.089454
potenciaGeradaTG2_1,2.347416e+13,1.937097,0.998533,0.122005
potenciaGeradaTG1_1,2.040903e-02,1.985988,0.997701,0.353621
vazaoVaporEscape,5.059453e-03,0.505734,0.998706,6.239934
consumoEspecificoTG1_1,1.581693e-02,1.554124,0.903524,18.697675
